## Function Definition

In [ ]:
def dwi_topup(brain_path,output_name,readout):
    '''
    readout: readout time for fist and second dwi, example readout=[0.157,0.157]
    '''
    # change slice number to even
    import os
    import numpy as np
    from dipy.io.image import load_nifti, save_nifti
    from dipy.segment.mask import applymask
    from dipy.io import read_bvals_bvecs
    from dipy.core.gradients import gradient_table
    from dipy.reconst.dti import TensorModel
    from dipy.reconst.dti import TensorFit
    from dipy.reconst.dti import color_fa
    from tqdm.notebook import tqdm
    from dipy.viz import regtools
    import dipy.denoise.noise_estimate as ne
    import nibabel as nib
    brain_path = brain_path
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        dwi_file1 = os.path.join(Study_folder,'AP.nii') # 4D diffusion data file name
        data1,affine1,img1 = load_nifti(dwi_file1,return_img=True)

        dwi_file2 = os.path.join(Study_folder,'PA.nii') # 4D diffusion data file name
        data2,affine2,img2 = load_nifti(dwi_file2,return_img=True)

        data_new = np.zeros(data1.shape[0:-1]+(2,))
        data_new[:,:,:,0] = data1[:,:,:,0]
        data_new[:,:,:,1] = data2[:,:,:,0]
        # img_new = nib.Nifti1Image(data_new.astype(np.int16), np.eye(4))
        # zooms_old = img1.header.get_zooms()[0:4]
        
        # save_nifti_origin(input=img_new.get_data(),output_name='AP_PA',type_output=np.int16,
        #                   zooms=zooms_old)
        save_nifti('AP_PA', data_new, affine1,hdr=img1.header)
        #######################
        readout_value1 = readout[0]
        readout_value2 = readout[1]
        acqparams = [[0,-1,0,readout_value1],[0,1,0,readout_value2]] #0.17646
        # index_number = 2
        index = [[1]] # index_number is the volume number of registration image #[[1,2]]
        np.savetxt('acqparams.txt',acqparams,fmt='%1.5f')
        np.savetxt('index.txt',index,fmt='%1.0f')
        ######################
        from nipype.interfaces.fsl.epi import TOPUP
        import nipype.interfaces.fsl as fsl
        topup = TOPUP()
        topup.inputs.in_file = os.path.join(Study_folder,'AP_PA.nii')
        topup.inputs.encoding_file = 'acqparams.txt'
        topup.inputs.output_type = 'NIFTI'
        topup.inputs.out_base = 'dwi_topup'
        topup.cmdline
        topup.run()
        #######################
        from nipype.interfaces.fsl import ApplyTOPUP
        applytopup = ApplyTOPUP()
        applytopup.inputs.in_files = [dwi_file1,dwi_file2]
        applytopup.inputs.encoding_file = 'acqparams.txt'
        applytopup.inputs.in_index = list([1,2])
        applytopup.inputs.in_topup_fieldcoef = 'dwi_topup_fieldcoef.nii'
        applytopup.inputs.in_topup_movpar = 'dwi_topup_movpar.txt'
        applytopup.inputs.output_type = "NIFTI"
        applytopup.inputs.method = 'jac' # 'jac'
        applytopup.inputs.out_corrected = output_name+'.nii'
        applytopup.run()

In [ ]:
readout_list = [0.03784,0.03784]
brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-01/TOPUP']
dwi_topup(brain_path,output_name='dwi_topuped',readout=readout_list)